In [ ]:
# | default_exp routes.publish

In [2]:
# | exporti
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda

# Publications

In [3]:
# | export
async def search_publications(auth: dmda.DomoAuth,
                              search_term: str = None, 
                              limit=100, offset=0,
                              session: httpx.AsyncClient = None, debug_api: bool = False) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/publication/summaries"

    params = {'limit': limit, 'offset': offset}

    if search_term:
        params.update({'searchTerm': search_term})

    res = await gd.get_data(auth=auth,
                         method='GET',
                         url=url,
                         params=params,
                         session=session,
                         debug_api=debug_api)

    return res


In [4]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", 
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await search_publications(auth=token_auth)

pd.DataFrame(res.response[0:5])


,id,name,description,dataSets,pages,cards,subscribers,totalSubscribers,invalidSubscribers,invalidDatasetSubscribers,invalidPageSubscribers,invalidCardSubscribers,isV2,hasDuplicate,refreshing,executionType,executionEstimatedCompletionTime,executionStartTime,executionEndTime
0,701eb547-4244-43f6-bcda-003ed2c483c7,test_publish,,1,0,0,0,1,0,0,0,0,True,False,False,NEW_PUBLICATION,1682581070465,1682580980470,1682580980950


In [6]:
# | export
async def get_publication_by_id(auth: dmda.DomoAuth,
                                publication_id: str,
                                session: httpx.AsyncClient = None, debug_api: bool = False) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/publication/{publication_id}"

    res = await gd.get_data(auth=auth,
                            method='GET',
                            url=url,
                            session=session,
                            debug_api=debug_api)

    return res


# generate publish body


In [7]:
import os
import pandas as pd

publication_id = "701eb547-4244-43f6-bcda-003ed2c483c7"

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_publication_by_id(publication_id = publication_id,
                                  auth=token_auth)

res.response


{'id': '701eb547-4244-43f6-bcda-003ed2c483c7',
 'name': 'test_publish',
 'customerId': 'mmmm-0012-0200',
 'created': 1682580980642,
 'content': {'id': 'dc7b65dd-28fc-4fa6-b469-26873c668549',
  'domain': 'domo-community.domo.com',
  'customerId': 'mmmm-0012-0200',
  'userId': '612085674',
  'type': 'PACKAGE',
  'updated': 1682580980642,
  'hash': 'cc90e85e-d3d6-43a3-86d7-46d9d7171272',
  'publicationId': '701eb547-4244-43f6-bcda-003ed2c483c7'},
 'description': '',
 'isV2': True,
 'hasDuplicate': False,
 'useDirectContent': True,
 'type': 'CONTENT',
 'isBackup': False,
 'children': [{'id': 'f80ef8c8-a133-439c-b459-7967ce7df4b3',
   'customerId': '',
   'created': 1683037775649,
   'content': {'id': '7234b90b-09d9-48cb-876a-d8ed11fc6c2a',
    'domain': 'domo-community.domo.com',
    'customerId': '',
    'userId': '612085674',
    'domoObjectId': 'e17fd398-8abf-4cb5-8ed1-5394e274fbf4',
    'type': 'DATASET',
    'updated': 1683037775649,
    'hash': '470f51ce-23d1-4c14-b077-1f97668d7437'}

In [10]:
#| export
def generate_publish_body(url: str, sub_domain_ls: [str], content_ls: [str], name: str, description: str, unique_id: str, is_new: bool):
    if not sub_domain_ls:
        sub_domain_ls = []

    if not content_ls:
        content_ls = []

    body = {
        "id": unique_id,
        "name": name,
        "description": description,
        "domain": url,
        "content": content_ls,
        "subscriberDomain": sub_domain_ls,
        "new": str(is_new).lower()
    }

    return body

In [11]:
import os
import pandas as pd

publication_id = "701eb547-4244-43f6-bcda-003ed2c483c7"

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)
body = generate_publish_body("domo-community.domo.com",["test.domo.com"],["content"],"test",'','id',True)
body

{'id': 'id',
 'name': 'test',
 'description': '',
 'domain': 'domo-community.domo.com',
 'content': ['content'],
 'subscriberDomain': ['test.domo.com'],
 'new': 'true'}

In [ ]:
#| export
# Creating publish job for a specific subscriber
async def create_publish_job(auth: dmda.DomoAuth, body : dict, session: httpx.AsyncClient = None, debug_api: bool = False) -> rgd.ResponseGetData:
    url = f'https://{auth.domo_instance}.domo.com/api/publish/v2/publication'

    if (debug_api):
        print (body)
    res = await gd.get_data(auth=auth,
                         method='POST',
                         url=url,
                         body=body,
                         session=session,
                         debug_api=debug_api)

    return res




In [ ]:
#| export
# Updating existing publish job with content
async def udpate_publish_job(auth: dmda.DomoAuth, publication_id: str, body: dict, session: httpx.AsyncClient = None, debug_api: bool = False) -> rgd.ResponseGetData:

    url = f'https://{auth.domo_instance}.domo.com/api/publish/v2/publication/{publication_id}'

    res = await gd.get_data(auth=auth,
                         method='PUT',
                         url=url,
                         body=body,
                         session=session,
                         debug_api=debug_api)
    return res

# # finds all jobs waiting to be accepted within the subscriber

# Subscriptions


In [ ]:
# | export
async def get_subscription_summaries(
    auth: dmda.DomoAuth, session: httpx.AsyncClient = None, debug_api: bool = False
) -> rgd.ResponseGetData:

    """retrieves a summary of existing subscriptions"""
    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/subscription/summaries"

    res = await gd.get_data(
        auth=auth, method="GET", url=url, session=session, debug_api=debug_api
    )
    return res

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_subscription_summaries(auth=token_auth)

pd.DataFrame(res.response[0:5])

,subscriptionId,publicationId,publisherDomain,publicationSummary,status,result,description,datasets,pages,cards,created,updated
0,a825f210-77cf-4313-a73d-4d2ae7cd3397,c7119a3e-f97d-4221-83c6-56ddd38073b0,datamavenllc-io.domo.com,"{'id': 'c7119a3e-f97d-4221-83c6-56ddd38073b0',...",READY,SUCCESS,Success.,4,2,42,1674166687000,1674166687000
1,3436f012-d9ff-4903-835e-3b88dcce11c9,d81b0a9a-e6f6-4ca0-b170-c57f91e75c22,datamavenllc-io.domo.com,"{'id': 'd81b0a9a-e6f6-4ca0-b170-c57f91e75c22',...",READY,MISSING_PAGE,"Missing page for content id, 465577116. Missin...",8,3,42,1645671430000,1674168501000


In [ ]:
# | export
async def get_subscription_invititations(
    auth: dmda.DomoAuth, session: httpx.AsyncClient = None, debug_api: bool = False
) -> rgd.ResponseGetData:
    """retrieves a list of subscription invitations"""

    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/subscription/invites"

    res = await gd.get_data(
        auth=auth, method="GET", url=url, session=session, debug_api=debug_api
    )
    return res

#### sample implementation of get_subscription_invitations

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_subscription_invititations(auth=token_auth)

pd.DataFrame(res.response[0:5])

,publisherDomain,publicationSummary,status
0,datamavenllc-io.domo.com,"{'id': '0f576510-f489-49a5-a2d0-d416221934ca',...",READY
1,datamavenllc-io.domo.com,"{'id': '6c0c8f80-5eda-4eef-8267-6875ccd596fb',...",READY


In [ ]:
# | export
async def accept_invite_by_id(auth: dmda.DomoAuth,
                              subscription_id: str, 
                              session: httpx.AsyncClient = None, debug_api: bool = False) -> rgd.ResponseGetData:
    """this takes get_subscription_invites_list into account and accepts - not instant"""

    url = f'https://{auth.domo_instance}.domo.com/api/publish/v2/subscription/{subscription_id}'

    res = await gd.get_data(auth=auth,
                        method='POST',
                        url=url,
                        session=session,
                        debug_api=debug_api)
    return res

In [ ]:
# | export
async def refresh_publish_jobs(auth: dmda.DomoAuth,
                               publish_ids: list,
                               session: httpx.AsyncClient = None, debug_api: bool = False) -> rgd.ResponseGetData:
    """Refreshing list of publish jobs. Typically "instance" = publisher instance"""

    url = f'https://{auth.domo_instance}.domo.com/api/publish/v2/publication/refresh'

    body = {
        'publicationIds': publish_ids
    }

    res = await gd.get_data(auth=auth,
                         method='PUT',
                         url=url,
                         body=body,
                         session=session,
                         debug_api=debug_api)
    return res


In [ ]:
#| hide

import nbdev
nbdev.nbdev_export()